<a href="https://colab.research.google.com/github/AndoorAlanD/DA6401-Assignment-1/blob/main/Question_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [3]:
#loading the dataset
from keras.datasets import fashion_mnist
from sklearn.model_selection import train_test_split
(X_train, y_train), (X_test, y_test) = fashion_mnist.load_data()
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.1)
class_names= ['T-shirt', 'Trouser','Pullover','Dress','Coat','Sandal','Shirt','Sneaker','Bag','Ankle boot']
X_train = X_train.reshape(X_train.shape[0], 28*28) / 255
X_test = X_test.reshape(X_test.shape[0], 28*28) /255
X_val = X_val.reshape(X_val.shape[0], 28*28) /255

29515/29515 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
26421880/26421880 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
5148/5148 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step
4422102/4422102 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


In [24]:
class Feed_Forward_Neural_Network():

  def __init__(self, config):
    self.theta = []
    self.W = []
    self.b = []

    self.config = config
    self.eta = self.config['lr']

    self.l = 2
    self.L = []
    self.L.append(X_train.shape[1])


    self.l += self.config['num_layers']
    for i in range(self.l-2):
      self.L.append(self.config['hidden_size'])
    self.L.append(np.max(y_train)+1)
    print(self.L)


    for i in range(self.l-1):
      self.W.append(np.random.randn(self.L[i+1], self.L[i]))
      self.b.append(np.random.randn(self.L[i+1]))

    if self.config['activation'] == 'sigmoid':
      self.act = self.sigmoid
      self.act_der = self.sigmoid_der
    elif self.config['activation'] == 'ReLU':
      self.act = self.relu
      self.act_der = self.relu_der
    else:
      self.act = self.identity
      self.act_der = self.identity_der

    self.optimizer = self.classic_gd(X_train, y_train)


  def identity(self, x):
    return x

  def identity_der(self, x):
    return np.ones(len(x))

  def sigmoid(self, x):
    return 1 / (1 + np.exp(-np.clip(x, -500, 500)))

  def sigmoid_der(self, x):
    sg = self.sigmoid(x)
    return sg * (1 - sg)

  def relu(self, x):
    return np.maximum(0, x)

  def relu_der(self, x):
    return (x > 0).astype(int)

  def softmax(self, x):
    new_x =np.exp(x - np.max(x))
    # new_x = np.exp(np.clip(x,-500, 500))
    return  new_x / new_x.sum()


  # def data_log(self, X=X_train, y=y_train):
  #   val_acc, val_true, val_pred = self.accuracy()
  #   train_acc, train_true, train_pred = self.accuracy(X, y)
  #   # val_acu=self.accuracy()
  #   # train_acu=self.accuracy(X,y)
  #   val_loss=self.cross_entropy_loss()
  #   train_loss=self.cross_entropy_loss(X,y)
  #   print(val_acu)
  #   wandb.log({
  #       'val_accuracy': val_acc,'val_loss': val_loss,'accuracy': train_acc,'loss': train_loss,
  #       'confusion_matrix': wandb.plot.confusion_matrix(probs=None,
  #                                                       y_true=val_true,
  #                                                       preds=val_pred,
  #                                                       cl ass_names=[str(i) for i in range(np.max(y) + 1)])
  #   })
  #   # wandb.log({'val_accuracy': val_acu,'val_loss': val_loss,'accuracy': train_acu,'loss': train_loss})

  # # def accuracy(self, X=X_val, y=y_val):
  # #   correct_predictions = sum(np.argmax(self.feed_forward(x)[-1]) == y_true for x, y_true in zip(X, y))
  # #   return (correct_predictions / len(y)) * 100.0

  # def accuracy(self, X=X_val, y=y_val):
  #   y_pred_labels = []
  #   y_true_labels = []

  #   for x, y_true in zip(X, y):
  #       y_pred = np.argmax(self.feed_forward(x)[-1])
  #       y_pred_labels.append(y_pred)
  #       y_true_labels.append(y_true)

  #   correct_predictions = sum(y_p == y_t for y_p, y_t in zip(y_pred_labels, y_true_labels))
  #   acc = (correct_predictions / len(y)) * 100.0

  #   return acc, y_true_labels, y_pred_labels

  def data_log(self, X=X_train, y=y_train):
    val_acc, val_true, val_pred = self.accuracy()
    train_acc, train_true, train_pred = self.accuracy(X, y)
    val_loss = self.cross_entropy_loss()
    train_loss = self.cross_entropy_loss(X, y)

    print("Validation Accuracy:", val_acc)
    print("Validation Loss:", val_loss)
    print("Training Accuracy:", train_acc)
    print("Training Loss:", train_loss)

    # Print confusion matrix
    num_classes = np.max(y) + 1
    print("Confusion Matrix:")
    confusion_matrix = np.zeros((num_classes, num_classes), dtype=int)

    for true, pred in zip(val_true, val_pred):
      confusion_matrix[true, pred] += 1

    print(confusion_matrix)

  def accuracy(self, X=X_val, y=y_val):
    y_pred_labels = []
    y_true_labels = []

    for x, y_true in zip(X, y):
      y_pred = np.argmax(self.feed_forward(x)[-1])
      y_pred_labels.append(y_pred)
      y_true_labels.append(y_true)

    correct_predictions = sum(y_p == y_t for y_p, y_t in zip(y_pred_labels, y_true_labels))
    acc = (correct_predictions / len(y)) * 100.0

    return acc, y_true_labels, y_pred_labels


  def squared_loss(self, X=X_val, y=y_val):
    sq_loss = 0
    for (x_in, y_true) in zip(X, y):
      _, _, y_out = self.feed_forward(x_in)
      # y_out[y_true] -= 1
      # sq_loss += np.sum(y_out**2)
      y_diff = y_out.copy()  # Avoid modifying original `y_out`
      y_diff[y_true] -= 1
      sq_loss += np.sum(y_diff**2)
    return sq_loss / len(y)

  def cross_entropy_loss(self, X=X_val, y=y_val):
    loss = 0
    for (x_in, y_true) in zip(X, y):
      a, h, y_out = self.feed_forward(x_in)
      loss -= np.log(y_out[y_true] + 1e-10)
    return loss / len(y)


  def feed_forward(self, x):
    pre_a = []
    act_h = [x]

    for i in range(self.l-2):
      pre_a.append(np.dot(self.W[i], act_h[i]) + self.b[i])
      act_h.append(self.act(pre_a[-1]))
    pre_a.append(self.b[-1] + np.dot(self.W[-1], act_h[-1]))
    y_pred = self.softmax(pre_a[-1])
    return pre_a, act_h, y_pred

  def back_prop(self, a, h, y, y_pred):
    grad_a = []

    if(self.config['loss'] == 'squared'):
      y_p = y_pred * (1-y_pred)
      y_pred[y] -= 1
      grad_a = np.array(y_p * y_pred)
    else:
      for i in range(len(y_pred)):
        if(i == y):
          grad_a.append(y_pred[i]-1)
        else:
          grad_a.append(y_pred[i])
      grad_a = np.array(grad_a)

    i = self.l-1
    grad_W, grad_b = [], []
    while i>0:
      grad_W.append(np.outer(grad_a, np.array(h[i-1])))
      grad_b.append(grad_a)
      if(i>1):
        grad_h = np.matmul(self.W[i-1].T, grad_a)
        grad_a = np.multiply(grad_h, self.act_der(a[i-2]))
      i -= 1

    return grad_W, grad_b

  def d_init(self):
    dw = [np.zeros_like(w) for w in self.W]
    db = [np.zeros_like(bi) for bi in self.b]
    return dw, db

  def classic_gd(self, X=X_train, y=y_train):
    max_epochs = self.config['epochs']
    index = 0

    for ep in range(max_epochs):
      print(f"Epoch {ep+1}/{max_epochs}")
      dw, db = self.d_init()

      for (x, y_true) in zip(X, y):
        a, h, y_pred = self.feed_forward(x)
        grad_W, grad_b = self.back_prop(a, h, y_true, y_pred)

        for i in range(self.l-1):
          dw[i] += grad_W[-1-i]
          db[i] += grad_b[-1-i]

      for i in range(self.l-1):
        self.W[i] -= self.eta*dw[i]
        self.b[i] -= self.eta*db[i]
      dw, db = self.d_init()

      self.data_log()


In [25]:
config = {
    'lr': 0.01,
    'num_layers': 3,
    'hidden_size': 64,
    'epochs': 5,
    'activation': 'ReLU',
    'loss': 'cross_entropy'
}
#cross_entropy
#squared

In [26]:
model = Feed_Forward_Neural_Network(config)
model.classic_gd(X_train, y_train)